In [ ]:
#Imports
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from torch.utils.data import random_split, DataLoader
from transformers import AutoTokenizer
from transformers import AutoTokenizer, BertForSequenceClassification, AdamW
from torch.nn.functional import cross_entropy
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW
import torch.nn.functional as F


In [ ]:
# Chinese Web Text 2.0 Dataset from Hugging face
ds = load_dataset("CASIA-LM/ChineseWebText2.0")

In [3]:
for i in range(5):
    print(ds['test'][i])

print(ds['test'].features)


print(len(ds['test']))


{'text': '为了您的交易安全,建议您选择诚信商或供求商会员进行贸易为了您的交易安全,建议您选择诚信商或供求商会员进行贸易企业名称: 深圳市嘉应锡业制品厂 企业类型:个人独资企业 经营模式:生产/制造 所 在 地:广东 深圳市 联 系 人:刘明 发布日期:2006-11-11 9:50:51 有效期至:2006-12-11 9:51:001 :本产品不燃不爆 ,无毒 ,无刺激性气体.2 :焊后易清洗,可用水洗工艺.3 :可用于手浸,涂刷等焊接工艺.', 'domain': {'single_label': 'general', 'multi_label': ['general']}, 'toxicity': {'label': 0, 'score': 1.0167443178943358e-05}, 'quality_score': 0.218505859375}
{'text': '广东省湛江市赤坎区人民法院民 事 裁 定 书(2018)粤0802民初3253号原告中国银行股份有限公司湛江分行,住所地:湛江市人民大道北71号.负责人盘仲廷,该分行行长.委托代理人杨国兴,男,广东东方昆仑律师事务所律师.委托代理人刘浩海,男,中国银行股份有限公司湛江分行员工.被告张捷军,男,1970年12月13日出生,汉族,住广东省湛江市霞山区,原告中国银行股份有限公司湛江分行诉被告张捷军金融借款合同纠纷一案,本院于2018年11月9日受理后,并于当天向原告送达了《预交受理费通知书》,明确告知其应于收到该通知书后7日内向本院预交案件受理费918.8元,但在该期限内原告未向本院预交上述费用,亦未向本院提交缓交申请.据此,依照《最高人民法院关于适用<中华人民共和国民事诉讼法>的解释》第二百一十三条之规定,裁定如下:本案按自动撤诉处理.审判员  杨莹莹二〇一八年十一月二十七日书记员  黄嘉慧', 'domain': {'single_label': 'law', 'multi_label': ['law']}, 'toxicity': {'label': 0, 'score': 2.043940003204625e-05}, 'quality_score': 0.98388671875}
{'text': '法学院开展"疫情中的热点法律问题研讨"学术沙龙为加强学术氛围,探索教育发展,

In [ ]:
learning_rate = 1e-4
nepochs = 20
batch_size = 32
max_length = 128



50
13


In [ ]:

#Importing BERT chinese tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")
text = "今天天气很好"
tokens = tokenizer.tokenize(text)
ids = tokenizer.convert_tokens_to_ids(tokens)

print("Tokens:", tokens)
print("Token IDs:", ids)


def yield_tokens(data_loader):
    for _, text in data_loader:
        yield tokenizer(text)

Tokens: ['今', '天', '天', '气', '很', '好']
Token IDs: [791, 1921, 1921, 3698, 2523, 1962]


In [ ]:
#The categories of the texts we wants to classify
categories = ["Book", "Dialogue", "Education", "Encyclopedia", "Law", "Math", "Medicine", "Finance", "News", "General", "Technology"]

def label_to_vector(labels, categories):
    vector = [1 if label in labels else 0 for label in categories]
    return vector

multi_labels = ["News", "Education"]
label_vector = label_to_vector(multi_labels, categories)
print(label_vector) 

[0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0]


In [ ]:

# My model for classifying texts
class CustomBERTClassifier(nn.Module):
    def __init__(self, num_labels):
        super(CustomBERTClassifier, self).__init__()
        self.bert = AutoModel.from_pretrained("bert-base-chinese")
        self.fc1 = nn.Linear(768, 256)
        self.fc2 = nn.Linear(256, num_labels)  
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)

    def forward(self, input_ids, attention_mask):
        output = self.bert(input_ids, attention_mask=attention_mask)
        cls_embedding = output.last_hidden_state[:, 0, :]  
        x = self.relu(self.fc1(cls_embedding))
        x = self.dropout(x)
        x = self.fc2(x)  
        return x  

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CustomBERTClassifier(num_labels=len(categories)).to(device)
optimizer = AdamW(model.parameters(), lr=learning_rate)
criterion = nn.BCEWithLogitsLoss()  


model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

In [ ]:
train_size = int(0.8 * len(ds['test'])) 
test_size = len(ds['test']) - train_size 
 
train_dataset, test_dataset = random_split(ds['test'], [train_size, test_size])

def tokenize_and_pad(batch):
    return tokenizer(batch['text'], padding=True, truncation=True, max_length=max_length, return_tensors="pt")

# Apply the tokenization to the dataset using DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=tokenize_and_pad)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, collate_fn=tokenize_and_pad)

print(len(train_loader))
print(len(test_loader))

50
13


In [ ]:
model.eval() 
test_loss = 0.0
correct_predictions = 0
total_predictions = 0

with torch.no_grad():  
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        
       
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits
        
        test_loss += loss.item()
        
        
        _, predicted = torch.max(logits, dim=1)
        correct_predictions += (predicted == labels).sum().item()
        total_predictions += labels.size(0)

avg_test_loss = test_loss / len(test_loader)
test_accuracy = correct_predictions / total_predictions * 100

print(f"Test Loss: {avg_test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")

TypeError: list indices must be integers or slices, not str

In [ ]:
# Évaluation
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        correct += (outputs.logits.argmax(dim=1) == labels).sum().item()
        total += labels.size(0)

print(f"Test Accuracy: {correct/total:.4f}")
